In [5]:
# Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import re
import time

In [7]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Set the executable path and initialize the firefox browser in splinter
executable_path = {'executable_path': 'geckodriver.exe'}
browser = Browser('firefox', **executable_path)

## Visit the NASA mars news site

In [4]:
# Visit the mars nasa news site
mars_url = 'https://mars.nasa.gov/news/'
browser.visit(mars_url)

# Set up optional delay for loading the page
# Verify if the element is present in the current page by CSS, and wait for 1 second

browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [5]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
mars_news_soup = BeautifulSoup(html, 'html.parser')

slide_element = mars_news_soup.select_one('ul.item_list li.slide')

In [6]:
# Use .find() the content title and save it as `content_title'

slide_element.find("div", class_='content_title')

<div class="content_title"><a href="/news/8959/nasas-insight-mars-lander-gets-a-power-boost/" target="_self">NASA's InSight Mars Lander Gets a Power Boost</a></div>

In [7]:
# # From the parent element find the first a tag and save it as `mars_news_title`
mars_news_title = slide_element.find("div", class_='content_title').get_text()
mars_news_title

"NASA's InSight Mars Lander Gets a Power Boost"

In [8]:
# .find() the paragraph text. paragraph text is in parent element
mars_news_para = slide_element.find('div', class_="article_teaser_body").get_text()
mars_news_para

'The spacecraft successfully cleared some dust off its solar panels, helping to raise its energy and delay when it will need to switch off its science instruments.'

## JPL Space Images Featured Image

In [13]:
# Got to image url
img_url = 'https://www.jpl.nasa.gov/images?query=mars'
browser.visit(img_url)

In [17]:
# Find and click the full image button
full_img_element = browser.find_by_css('img.BaseImage')[0]
full_img_element

In [19]:
image_url = full_img_element["src"]

image_url

'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA15964.2e16d0ba.fill-400x400-c50.jpg'

### **_The actual image url given for this work did not exist(not working) at the time of writing the script. A new url was used._** 

## Mars Weather

In [20]:
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

time.sleep(4)

In [21]:
html = browser.html
mars_weather_soup = BeautifulSoup(html, 'html.parser')

In [22]:
# Begin with a tweet with the data-name `Mars Weather`
weather_tweet_results = mars_weather_soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})

In [23]:

try:
    mars_weather = weather_tweet_results.find("p", "tweet-text").get_text()
    mars_weather

except AttributeError:

    pattern = re.compile(r'sol')
    mars_weather = mars_weather_soup.find('span', text=pattern).text
    mars_weather

mars_weather

'Perseverance sol 106 (Jun 7, 2021), high -22.0ºC/-7.6ºF, low -84.1ºC/-119.4ºF, pressure at 7.61 hPa and steady, daylight 05:32:20-18:30:30'

## Mars Hemispheres

In [24]:
# visit the USGS astrogeology page for hemisphere data from Mars

hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)

In [25]:
# create a variable called hem_img_urls and set it to an empty list

hem_image_urls = []

# Get a list of all of the hemispheres
links = browser.find_by_css("a.product-item h3")

# loop through links, click the link, find the sample anchor, return the href
for index in range(len(links)):
    hemisphere = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[index].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    hem_image_urls.append(hemisphere)
    
    # On completion navigate backwards
    browser.back()
    

C:\anaconda\lib\site-packages\splinter\driver\webdriver\__init__.py:498: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  warnings.warn(


In [26]:
hem_image_urls

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

## Mars Facts

In [27]:
#Scraping the table and getting HTML string and saving a dataframe in a variable 'facts_df'

import pandas as pd
# go to url and save the first element of the list to variable facts_df 
facts_df = pd.read_html('https://space-facts.com/mars/')[0]

facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [28]:
# Rename the columns
facts_df.columns=['Parameter', 'Value']

# Set parameter column as index
facts_df.set_index('Parameter', inplace=True)
facts_df

,Value
Parameter,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [29]:
facts_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Parameter</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n 

In [30]:
browser.quit()